In [1]:
import numpy as np

In [2]:
class territory:
    """A territory within Risk"""
    
    def __init__(self, name, continent, index):
        self.name = name
        self.continent = continent
        self.index = index
        
        self.owner = None
        self.num_units = 0
        return None
        
    def assign_owner(self, owner, num_units):
        self.owner = owner
        self.num_units = num_units
        return None
    
    def change_units(self, unit_addition):
        self.num_units += unit_addition
        return self.num_units

In [3]:
class player:
    """A player. This is the base class for the opponents and the agent."""
    
    def __init__(self): #Add these later if necessary: num_units=0, num_territories=0, continents_held=None):
        # Initialise all properties
        self.num_units = 0
        self.num_territories = 0
        self.continents = np.array([0, 0, 0])
        self.owned_territories = []
        return None
    
    def change_units(self, unit_addition):
        """Alter number of units"""
        self.num_units += unit_addition
        return self.num_units
    
    def add_territory(self, territory_index):
        self.num_territories +=1
        self.owned_territories.append(territory_index)
        return self.num_territories, self.owned_territories

In [4]:
class world_map:
    """The details of the Risk game board in it's current state"""
    
    def __init__(self, tot_players, tot_territories):
        self.tot_players = tot_players
        
        self.territory_index_lookup = {'Yakutsk':0,'Kamchatka':1,'Siberia':2,'Irkutsk':3,'Japan':4,
                          'Mongolia':5,'Ural':6,'China':7,'Afghanistan':8,'Middle East':9,
                          'India':10,'Siam':11,'Egypt':12,'North Africa':13,'East Africa':14,
                          'Congo':15,'Madagascar':16,'South Africa':17,'Indonesia':18,'New Guinea':19,
                          'Western Australia':20,'Eastern Australia':21}
        
        self.territory_connections = {0:[1,2,3],1:[0,3,4],2:[0,3,5,6,7],3:[0,1,2,5],4:[1,5],
                                      5:[2,3,4,7],6:[2,7,8],7:[2,5,6,8,10,11],8:[6,7,9,10],
                                      9:[8,10,12,14],10:[7,8,9,11],11:[7,10,18],12:[9,13,14],
                                      13:[12,14,15],14:[9,12,13,15,16,17],15:[13,14,17],16:[14,17],
                                      17:[14,15,16],18:[11,19,20],19:[18,20,21],20:[18,19,21],
                                      21:[19,20]}
        
        self.territory_name_lookup = {v: k for k, v in self.territory_index_lookup.items()}
        
        self.tot_territories = len(self.territory_index_lookup.keys())
        
        self.territories = self.reset_board()
        

    
    def reset_board(self):
        """Resets the game board and prepare for a new game of Risk
            Adds all territories to a single list whilst initialising them as classes"""
        self.territories = []
        for t_index in self.territory_name_lookup:
            # Determine continent
            if t_index <= 11:
                # Territory is in Asia
                continent = 0
            elif t_index <= 17:
                # Territory is in Africa
                continent = 1
            elif t_index <= 21:
                # Territory is in Oceana
                continent = 2
            else:
                raise "Too many territories input"

            self.territories.append(territory(self.territory_name_lookup[t_index], continent, t_index))
            
        return self.territories

    def find_connection_names(self, name=None, index=None):
        """Take in territory name or index as input and output the names of its neighbours"""
        if name == None:
            if index == None:
                raise "At least one of 'name' and 'index' must be input"
            # Use index to find connections
            connections = self.territory_connections[index]

            for i in range(len(connections)):

                connections[i] = self.territory_name_lookup[connections[i]]

            return connections
        else:
            # Use name to find connections
            connections = self.territory_connections[self.territory_index_lookup[name]]

            for i in range(len(connections)):

                connections[i] = self.territory_name_lookup[connections[i]]

            return connections
    
    def find_connection_indices(self, name=None, index=None):
        """Take in territory name or index as input and output the indices of its neighbours"""
        if name == None:
            if index == None:
                raise "At least one of 'name' and 'index' must be input"
            
            connections = self.territory_connections[index]

            return connections
        else:
            connections = self.territory_connections[territory_index_lookup[name]]

            return connections

In [5]:
class risk():
    
    def __init__(self,agent):
        
        self.players = [agent,player(),player()]
        self.board = world_map(3,22)
        

In [6]:
game = risk(0)

In [11]:
game.board.territories[0].name

'Yakutsk'

<h2>More complicated features</h2>
Note:
- These are complicated and may therefore be time consuming to calculate but they are similar to heuristics I have used whilst playing the game myself.
- These methods can be used on other players besides the learner so that the it can approximate the strength its opponents.

<h3>F1: Mean distance between owned countries</h3>

Note: Distance is measured such that adjacent territories have distance 1 between them, i.e. measured in edges.

Learning notes:
- Both measurements should measure the spread of territories. The hypothesis being that bunched territories are easier to defend.
    - The question is, which better estimates how well territories are bunched.
    - Separate but perhaps more useful, which is the best indicator of good position.
- Using mean distance should mean that sprawling map coverage is worse than clustered map coverage.
    - It will also take into account edge values more.
    - It may also dislike having multiple (separate) regions of owned territory, e.g. owning Australia and South Africa.
- Using median distance should mean that edge values are not used directly.

In [ ]:
def follow_path(world_map, node1, node2, path_so_far):
    """Recursive function for finding the shortest path between two nodes (in edges)"""
    if len(list(world_map.territory_connections[node1]))==0:
        # Terminating case: dead end
        return None
    elif len(list(world_map.territory_connections[node2]))==0:
        # Terminating case: dead end
        return None
    else:
        paths = []
        # Loop through connected nodes, finding the shortest path for each of them
        for n in world_map.territory_connections[node1]:
            if n in path_so_far:
                continue
            elif n == node2:
                return 1
            path = follow_path(n, node2, path_so_far+[n])
            if path != None:
                path+=1
                paths.append(path)
        if len(paths)==0:
            # Terminating case: all paths in this direction are dead ends
            return None
        else:
            return min(paths)
        
def shortest_path(world_map, node1, node2):
    """Find the shortest distance between two territories using a recursive function"""
    if not node1 in list(world_map.territory_connections.keys()):
        raise ValueError ('Territory {0] not recognised.'.format(node1))
    elif not node2 in list(world_map.territory_connections.keys()):
        raise ValueError ('Territory {0] not recognised.'.format(node2))
    else:
        shortest = follow_path(node1, node2, [])
        # Raising the error here allows the code to hit dead ends without causing undue errors
        if shortest == None:
            raise ValueError(str(('There is no path between node {0} and {1}'.format(node1, node2))))
        else:
            # Counting "hops"
            return shortest
        
def average_territory_distance(player, world_map):
    """Find the average distance (in edges) between all pairs of territories for a given player"""
    N = len(player.owned_territories)
    distances = []
    
    # Loop through all pairs of territories, calculating the shortest distance (in graph edges) between them
    for t in range(N): #territories
        t_index = player.owned_territories[t]
        for v in range(t+1, N):
            v_index = player.owned_territories[v]
            distances.append(shortest_path(world_map, t_index, v_index))
    mean_distance = np.mean(distances)
    median_distance = np.sort(distances)[int(len(distances) / 2)]
    return mean_distance, median_distance

<h3>F2: Strength of front line territories</h3>

Notes:
- Strength is determined by number of units on that territory
- Front line territories are defined as the ones directly adjacent to territories owned by other players

Learning notes:
- The distinction between front line and others will probably be more useful in the late game
    - It will be slow to calculate in early game but can be used then as an approximate metric for overall unit numbers
    
_Implementation notes:_
- _What is preferable, total units on front line or average? Both will be calculated and total will be returned for now_
- _I am also returning the minimum number of front-line units since that may also be useful._
    - _As an extension to this, I'm returning which territory has the lowest number of units._

In [ ]:
def strength_of_front_line(player, world_map):
    """Calculate the number of players in each territory adjacent to enemy territory"""
    N = len(player.owned_territories)
    min_front_line_units = np.inf
    
    # Loop through all player territory to determine whether it is in the front line
    for t in range(N): #territories
        t_index = player.owned_territories[t]
        
        is_front_line = False
        for at in world_map.territory_connections[t_index]: #adacent territories
            if at not in player.owned_territories:
                
                is_front_line = True
                break
        
        if is_front_line:
            t_num_units = self.territories[t_index].num_units
            front_line_units.append(t_num_units)
            
            if t_num_units < min_front_line_units:
                min_front_line_units = t_num_units
                weakest_link = t_index
    
    if min_front_line_units == np.inf:
        raise "This function has found no front line territories."
    
    tot_front_line_units = np.sum(front_line_units)
    mean_front_line_units = np.mean(front_line_units)
    
    return tot_front_line_units, min_front_line_units, weakest_link